In [2]:
import pygame
import random
import math

In [4]:
# Simple pygame program

# Import and initialize the pygame library
from pygame import mixer
pygame.init()

#Create the screen
screen = pygame.display.set_mode((800, 600))
#Title and icon
pygame.display.set_caption("Space invaders")
icon = pygame.image.load('ufo.png')
pygame.display.set_icon(icon)

#Background Image
background = pygame.image.load('background.png')
#Background music
mixer.music.load('background1.mp3')
mixer.music.play(-1)

#Player is a image which has a x and y coordinate
playerImg = pygame.image.load('player.png')
playerX = 300
playerY = 480
playerX_change = 0

#Enemy is a image which has a x and y coordinate
enemyImg = []
enemyX = []
enemyY = []
enemyX_change = []
enemyY_change = []

num_of_enemies = 6

for i in range(num_of_enemies):
    
    enemyImg.append(pygame.image.load('enemy.png'))
    enemyX.append(random.randint(0, 735))
    enemyY.append(random.randint(50, 150))
    enemyX_change.append(3)
    enemyY_change.append(40)

#Bullet is a image which has a x and y coordinate
bulletImg = pygame.image.load('bullet.png')
bulletX = 0
bulletY = 450
bulletX_change = 0
bulletY_change = 5
#state of bullet
#Ready - you can't see the bullet on the screen
#Fire - The bullet is moving
bullet_state = 'ready'

#Score
score_val = 0
font = pygame.font.Font('freesansbold.ttf', 32)

#Game over
over_font = pygame.font.Font('freesansbold.ttf', 64)

def show_score(x, y):
    score = font.render("SCORE: "+ str(score_val), True, (0, 255, 0))
    screen.blit(score, (x, y))

#this function is to draw the image on the surface(window) at a certain coordinate
def player(x, y):
    screen.blit(playerImg, (x, y))

def enemy(x, y, i):
    screen.blit(enemyImg[i], (x, y))

def fire_bullet(x, y):
    global bullet_state
    bullet_state = 'fire'
    screen.blit(bulletImg, (x+16, y+16))

def game_over_text():
    over_text = over_font.render("GAME OVER", True, (255, 255, 255))
    screen.blit(over_text, (200, 250))
#function to check whether a collision happened or not
def isCollision(enemyX, enemyY, bulletX, bulletY):
    distance = math.sqrt(math.pow(enemyX-bulletX, 2) + math.pow(enemyY-bulletY, 2))
    if distance < 27: return True
    return False

#running is to check whether we clicked the close button on the window or not
running = True
while running:
    #fill color to screen
    screen.fill((0, 100, 0))
    #Background image
    screen.blit(background, (0, 0))
    #checks every event in pygame events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        # check if a keystroke is pressed or not
        if event.type == pygame.KEYDOWN:
            #print("A keystroke is pressed")
            if event.key == pygame.K_LEFT:
                playerX_change = -3
            if event.key == pygame.K_RIGHT:
                playerX_change = 3
            if event.key == pygame.K_SPACE and bullet_state == 'ready':
                bullet_sound = mixer.Sound('laser.wav')
                bullet_sound.play()
                bulletX = playerX
                fire_bullet(bulletX, bulletY)
        # check if a keystroke is released
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                playerX_change = 0
    
    playerX += playerX_change
    #Boundary check
    if playerX <= 0: playerX = 0
    elif playerX >= 736: playerX = 736
    
    for i in range(num_of_enemies):
        # Game over check
        if enemyY[i] > 440:
            for j in range(num_of_enemies):
                enemyY[j] = 2000
            game_over_text()
            break

        enemyX[i] += enemyX_change[i]
        if enemyX[i]<=0: 
            enemyX_change[i] = 2
            enemyY[i]+= enemyY_change[i]
        elif enemyX[i] >= 736: 
            enemyX_change[i] = -2
            enemyY[i] += enemyY_change[i]
        
        #collision check
        if(isCollision(enemyX[i], enemyY[i], bulletX, bulletY)):
            explosion_sound = mixer.Sound('explosion.wav')
            explosion_sound.play()
            bullet_state = 'ready'
            bulletY = 450
            score_val += 1
            enemyX[i] = random.randint(0, 735)
            enemyY[i] = random.randint(50, 150)
        enemy(enemyX[i], enemyY[i], i)
    #Bullet movement
    if bullet_state == 'fire':
        fire_bullet(bulletX, bulletY)
        bulletY -= bulletY_change
    if bulletY <= 0:
        bullet_state = 'ready'
        bulletY = 450
     
    player(playerX, playerY)
    show_score(10, 10)
    #update the screen so that the changes are shown 
    pygame.display.update()

pygame.quit()